In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using PyPlot
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface;
include("quadratic_cost.jl")
include("walker.jl")
include("utils.jl")

 Activating environment at `~/Classes/OptimalControl/hw4_solutions/Project.toml`
 Activating environment at `~/Classes/OptimalControl/hw4_solutions/Project.toml`


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4284601554884975413\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17812614365081700580\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4284601554884975413\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

slerp (generic function with 1 method)

# Q2: Hybrid Trajectory Optimization

In [2]:
"""
    HybridNLP{n,m,L,Q}

Represents a (N)on(L)inear (P)rogram of a trajectory optimization problem,
with a dynamics model of type `L`, a quadratic cost function, horizon `T`, 
and initial and final state `x0`, `xf`.

The kth state and control can be extracted from the concatenated state vector `Z` using
`Z[nlp.xinds[k]]`, and `Z[nlp.uinds[k]]`.

# Constructor
    NLP(model, obj, tf, T, x0, xf, [integration])

# Basic Methods
    Base.size(nlp)    # returns (n,m,T)
    num_ineq(nlp)     # number of inequality constraints
    num_eq(nlp)       # number of equality constraints
    num_primals(nlp)  # number of primal variables
    num_duals(nlp)    # total number of dual variables
    packZ(nlp, X, U)  # Stacks state `X` and controls `U` into one vector `Z`

# Evaluating the NLP
The NLP supports the following API for evaluating various pieces of the NLP:

    eval_f(nlp, Z)         # evaluate the objective
    grad_f!(nlp, grad, Z)  # gradient of the objective
    hess_f!(nlp, hess, Z)  # Hessian of the objective
    eval_c!(nlp, c, Z)     # evaluate the constraints
    jac_c!(nlp, c, Z)      # constraint Jacobian
"""
struct HybridNLP{n,m,L,Q} <: MOI.AbstractNLPEvaluator
    model::L                                 # dynamics model
    obj::Vector{QuadraticCost{n,m,Float64}}  # objective function
    T::Int                                   # number of knot points
    M::Int                                   # number of steps in each mode
    Nmodes::Int                              # number of modes
    tf::Float64                              # total time (sec)
    x0::MVector{n,Float64}                   # initial condition
    xf::MVector{n,Float64}                   # final condition
    times::Vector{Float64}                   # vector of times
    modes::Vector{Int}                       # mode ID
    xinds::Vector{SVector{n,Int}}            # Z[xinds[k]] gives states for time step k
    uinds::Vector{SVector{m,Int}}            # Z[uinds[k]] gives controls for time step k
    cinds::Vector{UnitRange{Int}}
    lb::Vector{Float64}
    ub::Vector{Float64}
    zL::Vector{Float64}
    zU::Vector{Float64}
    rows::Vector{Int}
    cols::Vector{Int}
    function HybridNLP(model, obj::Vector{<:QuadraticCost{n,m}},
            tf::Real, T::Integer, M::Integer, x0::AbstractVector, xf::AbstractVector, integration::Type{<:QuadratureRule}=RK4
        ) where {n,m}
        xinds = [SVector{n}((k-1)*(n+m) .+ (1:n)) for k = 1:T]
        uinds = [SVector{m}((k-1)*(n+m) .+ (n+1:n+m)) for k = 1:T-1]
        times = collect(range(0, tf, length=T))
        modes = map(1:T) do k
            isodd((k-1) ÷ M + 1) ? 1 : 2
        end
        Nmodes = Int(ceil(T/M))
        
        c_init_inds = 1:n                                                    # initial constraint
        c_term_inds = (c_init_inds[end]+1):(c_init_inds[end]+n)              # terminal constraint
        c_dyn_inds = (c_term_inds[end]+1):(c_term_inds[end]+n*(T-1)  )       # dynamics constraints
        c_stance_inds = (c_dyn_inds[end]+1):(c_dyn_inds[end]+T)              # stance constraint (1 per time step)
        c_length_inds = (c_stance_inds[end]+1):(c_stance_inds[end]+(2*T))    # length bounds     (2 per time step)
        m_nlp = c_length_inds[end]                                           # total number of constraints
        
        lb = fill(0.0,m_nlp)
        ub = fill(0.0,m_nlp)
        lb[c_length_inds] .= model.ℓ_min
        ub[c_length_inds] .= model.ℓ_max

        cinds = [c_init_inds, c_term_inds, c_dyn_inds, c_stance_inds, c_length_inds]
        
        n_nlp = n*T + (T-1)*m
        zL = fill(-Inf, n_nlp)
        zU = fill(+Inf, n_nlp)
        rows = Int[]
        cols = Int[]
        
        new{n,m,typeof(model), integration}(
            model, obj,
            T, M, Nmodes, tf, x0, xf, times, modes,
            xinds, uinds, cinds, lb, ub, zL, zU, rows, cols
        )
    end
end
Base.size(nlp::HybridNLP{n,m}) where {n,m} = (n,m,nlp.T)
num_primals(nlp::HybridNLP{n,m}) where {n,m} = n*nlp.T + m*(nlp.T-1)
num_duals(nlp::HybridNLP) = nlp.cinds[end][end]

"""
    packZ(nlp, X, U)

Take a vector state vectors `X` and controls `U` and stack them into a single vector Z.
"""
function packZ(nlp, X, U)
    Z = zeros(num_primals(nlp))
    for k = 1:nlp.T-1
        Z[nlp.xinds[k]] = X[k]
        Z[nlp.uinds[k]] = U[k]
    end
    Z[nlp.xinds[end]] = X[end]
    return Z
end

"""
    unpackZ(nlp, Z)

Take a vector of all the states and controls and return a vector of state vectors `X` and
controls `U`.
"""
function unpackZ(nlp, Z)
    X = [Z[xi] for xi in nlp.xinds]
    U = [Z[ui] for ui in nlp.uinds]
    return X, U
end

unpackZ

## Costs

In [4]:
"""
    eval_f(nlp, Z)

Evaluate the objective, returning a scalar.
"""
function eval_f(nlp::HybridNLP, Z)
    # TASK: compute the objective value (cost)
    J = 0.0
    
    # SOLUTION
    xi,ui = nlp.xinds, nlp.uinds
    for k = 1:nlp.T-1
        x,u = Z[xi[k]], Z[ui[k]]
        J += stagecost(nlp.obj[k], x, u)
    end
    J += termcost(nlp.obj[end], Z[xi[end]])
    return J
end

"""
    grad_f!(nlp, grad, Z)

Evaluate the gradient of the objective at `Z`, storing the result in `grad`.
"""
function grad_f!(nlp::HybridNLP{n,m}, grad, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    obj = nlp.obj
    for k = 1:nlp.T-1
        x,u = Z[xi[k]], Z[ui[k]]
        # TASK: Compute the cost gradient
        grad[xi[k]] .= 0
        grad[ui[k]] .= 0
        
        # SOLUTION
        grad[xi[k]] = obj[k].Q*x + obj[k].q
        grad[ui[k]] = obj[k].R*u + obj[k].r
    end
    grad[xi[end]] = obj[end].Q*Z[xi[end]] + obj[end].q
    return nothing
end

"""
    hess_f!(nlp, hess, Z)

Evaluate the Hessian of the objective at `Z`, storing the result in `hess`.
Should work with `hess` sparse.
"""
function hess_f!(nlp::HybridNLP{n,m}, hess, Z, rezero=true) where {n,m}
    # TASK: Compute the objective hessian
    # HINT: It's a diagonal matrix
    if rezero
        for i = 1:size(hess,1)
            hess[i,i] = 0
        end
    end
    xi,ui = nlp.xinds, nlp.uinds
    obj = nlp.obj
    i = 1
    for k = 1:nlp.T
        for j = 1:n
            hess[i,i] += nlp.obj[k].Q[j,j]
            i += 1
        end
        if k < nlp.T
            for j = 1:m
                hess[i,i] += nlp.obj[k].R[j,j]
                i += 1
            end
        end
    end
end

hess_f!

## Constraints

In [3]:
function dynamics_constraint!(nlp::HybridNLP{n,m}, c, ztraj) where {n,m}
    Nt = nlp.T
    Nx,Nu = n,m
    Nmodes = nlp.Nmodes
    model = nlp.model
    xi,ui = nlp.xinds, nlp.uinds
    Z = ztraj
    dt = nlp.times[2]
    Nm = nlp.M

    d = reshape(view(c, nlp.cinds[3]),Nx,Nt-1)
    for k = 1:(Nmodes-1)
        for j = 1:(Nm-1)
            s = (k-1)*Nm + j
            x,u = Z[xi[s]], Z[ui[s]]
            x2 = Z[xi[s+1]]
            if mod(k,2) == 1
                d[:,s] = stance1_dynamics_rk4(model, x, u, dt) - x2 
            else
                d[:,s] = stance2_dynamics_rk4(model, x, u, dt) - x2 
            end
        end
        s = k*Nm
        x,u = Z[xi[s]], Z[ui[s]]
        x2 = Z[xi[s+1]]
        if mod(k,2) == 1
            d[:,s] = jump2_map(stance1_dynamics_rk4(model, x,u, dt)) - x2
        else
            d[:,s] = jump1_map(stance2_dynamics_rk4(model, x,u, dt)) - x2
        end

    end
    for j = 1:(Nm-1)
        s = (Nmodes-1)*Nm + j
        x,u = Z[xi[s]], Z[ui[s]]
        x2 = Z[xi[s+1]]
        if mod(Nmodes,2) == 1
            d[:,s] = stance1_dynamics_rk4(model, x,u, dt) - x2
        else
            d[:,s] = stance2_dynamics_rk4(model, x,u, dt) - x2
        end
    end
    
    return nothing
end

function dynamics_jacobian!(nlp::HybridNLP{n,m}, jac, Z) where {n,m}
    D = view(jac, nlp.cinds[3], :)
    Nt = nlp.T
    Nx,Nu = n,m
    Nmodes = nlp.Nmodes
    model = nlp.model
    xi,ui = nlp.xinds, nlp.uinds
    dt = nlp.times[2]
    Nm = nlp.M

    ic = 1:n
    for k = 1:(Nmodes-1)
        for j = 1:(Nm-1)
            s = (k-1)*Nm + j
            x,u = Z[xi[s]], Z[ui[s]]
            zi = [xi[s]; ui[s]]
            F = view(D, ic, zi)
            F2 = view(D, ic, xi[s+1])

            if mod(k,2) == 1
                F .= stance1_jacobian(model, x, u, dt)
            else
                F .= stance2_jacobian(model, x, u, dt)
            end
            for i = 1:n
                F2[i,i] = -1
            end

            ic = ic .+ n
        end
        s = k*Nm
        x,u = Z[xi[s]], Z[ui[s]]
        zi = [xi[s]; ui[s]]
        F = view(D, ic, zi)
        F2 = view(D, ic, xi[s+1])
        if mod(k,2) == 1
            F .= jump2_jacobian()*stance1_jacobian(model, x,u,dt)
        else
            F .= jump1_jacobian()*stance2_jacobian(model, x,u,dt)
        end
        for i = 1:n
            F2[i,i] = -1
        end
        ic = ic .+ n
    end
    for j = 1:(Nm-1)
        s = (Nmodes-1)*Nm + j
        x,u = Z[xi[s]], Z[ui[s]]
        zi = [xi[s]; ui[s]]
        F = view(D, ic, zi)
        F2 = view(D, ic, xi[s+1])

        if mod(Nmodes,2) == 1
            F .= stance1_jacobian(model, x, u, dt)
        else
            F .= stance2_jacobian(model, x, u, dt)
        end
        for i = 1:n
            F2[i,i] = -1
        end
        ic = ic .+ n
    end
    
    return nothing
end


function stance_constraint!(nlp::HybridNLP{n,m}, c,ztraj) where {n,m}
    Nt = nlp.T
    Nx,Nu = n,m
    Nmodes = nlp.Nmodes
    Nm = nlp.M
    model = nlp.model
    xi,ui = nlp.xinds, nlp.uinds
    Z = ztraj

    d = view(c, nlp.cinds[4])
    # z = reshape(ztraj[1:(end-Nx)],Nx+Nu,Nt-1)
    # xtraj = [z[1:Nx,:] ztraj[(end-(Nx-1)):end]]
    t = 1
    for k = 1:Nmodes
        if mod(k,2) == 1
            for j = 1:Nm
                s = (k-1)*Nm + j
                x = Z[xi[s]]
                d[t] = x[4]  # keep foot on the floor
                t += 1
            end
        else
            for j = 1:Nm
                s = (k-1)*Nm + j
                x = Z[xi[s]]
                d[t] = x[6]
                t += 1
            end
        end
            
    end
    return d 
end

function length_constraint!(nlp::HybridNLP{n,m}, c, ztraj) where {n,m}
    Nt = nlp.T
    Nx,Nu = n,m
    Nmodes = nlp.Nmodes
    model = nlp.model
    xi = nlp.xinds
    Z = ztraj

    d = view(c, nlp.cinds[5])
    # z = reshape(ztraj[1:(end-Nx)],Nx+Nu,Nt-1)
    # xtraj = [z[1:Nx,:] ztraj[(end-(Nx-1)):end]]
    for k = 1:Nt
        x = Z[xi[k]]
        d[2*(k-1)+1] = norm(x[1:2] - x[3:4])
        d[2*(k-1)+2] = norm(x[1:2] - x[5:6])
    end
    return d
end

function eval_c!(nlp::HybridNLP, c, ztraj)
    # c[c_init_inds] .= ztraj[1:Nx] - xref[:,1] #initial state constraint
    # c[c_term_inds] .= ztraj[(end-(Nx-1)):end] - xref[:,end] #terminal state constraint
    xi = nlp.xinds
    c[nlp.cinds[1]] .= ztraj[xi[1]] - nlp.x0
    c[nlp.cinds[2]] .= ztraj[xi[end]] - nlp.xf
    dynamics_constraint!(nlp, c, ztraj)
    stance_constraint!(nlp, c, ztraj)
    length_constraint!(nlp, c, ztraj)
end


function jac_c!(nlp::HybridNLP{n,m}, jac, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    c = zeros(eltype(Z), num_duals(nlp))
    # ForwardDiff.jacobian!(jac, con!, c, Z)

    dynamics_jacobian!(nlp, jac, Z)

    jac[nlp.cinds[1], xi[1]] .= I(n)
    jac[nlp.cinds[2], xi[end]] .= I(n)
    jac_stance = view(jac, nlp.cinds[4], :)
    jac_length = view(jac, nlp.cinds[5], :)

    # c = zeros(size(jac,1))
    # jac_stance = zero(jac)
    # ForwardDiff.jacobian!(jac_stance, (c,x)->stance_constraint!(nlp,c,x), c, Z)
    # jac .+= jac_stance
    # return
    
    t = 1
    for k = 1:nlp.T
        x = Z[xi[k]]
        foot_ind = nlp.modes[k] == 1 ? 4 : 6
        jac_stance[t, xi[k][foot_ind]] = 1
        
        d1 = x[1:2] - x[3:4]
        d2 = x[1:2] - x[5:6]
        # if abs(norm(d1)) > 1e-10
        jac_length[2*(k-1)+1, xi[k][1:2]] = +d1 / norm(d1)
        jac_length[2*(k-1)+1, xi[k][3:4]] = -d1 / norm(d1)
        # end
        # if abs(norm(d2)) > 1e-10
        jac_length[2*(k-1)+2, xi[k][1:2]] = +d2 / norm(d2)
        jac_length[2*(k-1)+2, xi[k][5:6]] = -d2 / norm(d2)
        # end
        t += 1
    end
end

jac_c! (generic function with 1 method)

jacobian_structure (generic function with 1 method)

In [127]:
jac_struct = jacobian_structure(nlp)
colvec = matrix_colors(jac_struct)
@time FiniteDiff.finite_difference_jacobian!(jac2, (c,x)->eval_c!(nlp, c, x), Z0)
@time FiniteDiff.finite_difference_jacobian!(jac2, (c,x)->eval_c!(nlp, c, x), Z0, colorvec=colvec)

  0.580910 seconds (7.40 M allocations: 667.772 MiB, 13.04% gc time)
  0.150211 seconds (417.48 k allocations: 33.410 MiB, 24.89% gc time)


## Problem Definition

In [5]:
# Dynamics model
model = SimpleWalker()

# Discretization
tf = 4.4
dt = 0.1
T = Int(ceil(tf/dt)) + 1
M = 5
times = range(0,tf, length=T);

## Reference Trajectory

In [6]:
function reference_trajectory(model::SimpleWalker, times)
    n,m = size(model)
    tf = times[end]
    T = length(times)
    mb,g = model.mb, model.g
    uref = [0.5*mb*g; 0.5*mb*g; 0.0]
    xref = zeros(n,T)
    xref[1,:] .= LinRange(-1.5,1.5,T)
    xref[2,:] .= ones(T)
    xref[3,:] .= LinRange(-1.5,1.5,T)
    xref[5,:] .= LinRange(-1.5,1.5,T)
    xref[7,2:end-1] .= (3.0/tf)*ones(T-2)
    xref[9,2:end-1] .= (3.0/tf)*ones(T-2)
    xref[11,2:end-1] .= (3.0/tf)*ones(T-2);  
    Uref = [SVector{m}(uref) for k = 1:T-1]
    Xref = [SVector{n}(x) for x in eachcol(xref)]
    return Xref, Uref
end
Xref,Uref = reference_trajectory(model, times);

## Solve


In [7]:
# Objective
using Random
Random.seed!(1)
Q = Diagonal([1; 10; fill(1.0, 4); 1; 10; fill(1.0, 4)]);
R = Diagonal(fill(1e-3,3))
Qf = Q;
obj = map(1:T-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[T], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, tf, T, M, Xref[1], Xref[end]);

# Initial guess
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Z0 = packZ(nlp, Xguess, Uguess);

In [8]:
include("moi.jl")

solve (generic function with 1 method)

In [14]:
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   401184
Number of nonzeros in inequality constraint Jacobian.:    60480
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      672
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      597
Total number of inequality constraints...............:       90
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:       90
        inequality constraints with only upper bounds:       

([-1.4999999999960252, 0.9999999999985604, -1.5000000000065257, -2.1413819911744338e-15, -1.4999999999970604, 6.278999625766614e-13, 1.2908842446867993e-12, -8.118034546908155e-14, -9.496133985715675e-13, 2.1824239324226535e-13  …  1.5000000000003775, -1.6923521575823195e-14, 1.4999999999998659, -1.4282033857016296e-12, -1.3392622047649934e-13, 4.829102098910099e-13, -5.598214564578225e-14, 2.531050146471582e-14, 1.3627496849909076e-13, 5.812127858544501e-13], Ipopt.Optimizer)

## Visualizer

In [15]:
vis = Visualizer()
set_mesh!(vis, model)
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/brian/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("Rfoot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))),"torso" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("body" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x62, 0x6f, 0x64, 0x79], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"Laxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"Lleg" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x4c, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))),"Rleg" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x52, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))),"Raxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))),"Lfoot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x4c, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x4c, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))))), Set{Any}(), ip"127.0.0.1", 8702))

In [17]:
visualize!(vis, nlp, unpackZ(nlp, Z_sol)[1])